In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

RAW_PATH = "/content/drive/MyDrive/uidai_hackathon/data/raw"
os.listdir(RAW_PATH)


['api_data_aadhar_biometric_0_500000.csv',
 'api_data_aadhar_biometric_1500000_1861108.csv',
 'api_data_aadhar_biometric_1000000_1500000.csv',
 'api_data_aadhar_biometric_500000_1000000.csv',
 'api_data_aadhar_demographic_1000000_1500000.csv',
 'api_data_aadhar_demographic_2000000_2071700.csv',
 'api_data_aadhar_demographic_0_500000.csv',
 'api_data_aadhar_demographic_1500000_2000000.csv',
 'api_data_aadhar_demographic_500000_1000000.csv',
 'api_data_aadhar_enrolment_1000000_1006029.csv',
 'api_data_aadhar_enrolment_0_500000.csv',
 'api_data_aadhar_enrolment_500000_1000000.csv']

In [ ]:
import pandas as pd
import numpy as np
from glob import glob


In [ ]:
enrol_files = glob(f"{RAW_PATH}/*enrolment*.csv")
demo_files  = glob(f"{RAW_PATH}/*demographic*.csv")
bio_files   = glob(f"{RAW_PATH}/*biometric*.csv")

print("Enrolment files:", len(enrol_files))
print("Demographic files:", len(demo_files))
print("Biometric files:", len(bio_files))


Enrolment files: 3
Demographic files: 5
Biometric files: 4


In [ ]:
enrol = pd.concat([pd.read_csv(f) for f in enrol_files], ignore_index=True)
demo  = pd.concat([pd.read_csv(f) for f in demo_files], ignore_index=True)
bio   = pd.concat([pd.read_csv(f) for f in bio_files], ignore_index=True)

print(enrol.shape, demo.shape, bio.shape)


(1006029, 7) (2071700, 6) (1861108, 6)


In [ ]:
print("Enrolment columns:", enrol.columns.tolist())
print("Demographic columns:", demo.columns.tolist())
print("Biometric columns:", bio.columns.tolist())


Enrolment columns: ['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater']
Demographic columns: ['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17_']
Biometric columns: ['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_']


In [ ]:
for df in [enrol, demo, bio]:
    print(df["date"].isna().sum())


0
0
0


In [ ]:
enrol_agg = enrol.groupby(["date", "state"], as_index=False).sum()
demo_agg  = demo.groupby(["date", "state"], as_index=False).sum()
bio_agg   = bio.groupby(["date", "state"], as_index=False).sum()

print(enrol_agg.shape, demo_agg.shape, bio_agg.shape)


(3393, 7) (4339, 6) (3984, 6)


In [ ]:
# Ensure date dtype is consistent before merge
for df in [enrol_agg, demo_agg, bio_agg]:
    df["date"] = pd.to_datetime(df["date"], dayfirst=True)

# Now merge safely
merged = enrol_agg.merge(demo_agg, on=["date", "state"], how="left")
merged = merged.merge(bio_agg, on=["date", "state"], how="left")

merged.fillna(0, inplace=True)
merged.head()



,date,state,district_x,pincode_x,age_0_5,age_5_17,age_18_greater,district_y,pincode_y,demo_age_5_17,demo_age_17_,district,pincode,bio_age_5_17,bio_age_17_
0,2025-03-02,Meghalaya,East Khasi Hills,793121,11,61,37,0,0.0,0.0,0.0,0,0.0,0.0,0.0
1,2025-03-09,Bihar,SitamarhiSitamarhiPurbi ChamparanSitamarhiSita...,6723137,206,633,166,0,0.0,0.0,0.0,0,0.0,0.0,0.0
2,2025-03-09,Delhi,West DelhiWest Delhi,220077,122,53,57,0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,2025-03-09,Haryana,FaridabadGurugram,243005,98,79,23,0,0.0,0.0,0.0,0,0.0,0.0,0.0
4,2025-03-09,Karnataka,Bengaluru UrbanBengaluru UrbanBengaluru UrbanB...,2240159,63,80,105,0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [ ]:
merged["total_enrolment"] = (
    merged["age_0_5"] +
    merged["age_5_17"] +
    merged["age_18_greater"]
)

merged["demographic_updates"] = (
    merged["demo_age_5_17"] +
    merged["demo_age_17_"]
)

merged["biometric_updates"] = (
    merged["bio_age_5_17"] +
    merged["bio_age_17_"]
)

merged["DUI"] = merged["demographic_updates"] / merged["total_enrolment"]
merged["BUBI"] = merged["biometric_updates"] / (
    merged["age_5_17"] + merged["age_18_greater"]
)

merged.replace([np.inf, -np.inf], 0, inplace=True)
merged.head()


,date,state,district_x,pincode_x,age_0_5,age_5_17,age_18_greater,district_y,pincode_y,demo_age_5_17,demo_age_17_,district,pincode,bio_age_5_17,bio_age_17_,total_enrolment,demographic_updates,biometric_updates,DUI,BUBI
0,2025-03-02,Meghalaya,East Khasi Hills,793121,11,61,37,0,0.0,0.0,0.0,0,0.0,0.0,0.0,109,0.0,0.0,0.0,0.0
1,2025-03-09,Bihar,SitamarhiSitamarhiPurbi ChamparanSitamarhiSita...,6723137,206,633,166,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1005,0.0,0.0,0.0,0.0
2,2025-03-09,Delhi,West DelhiWest Delhi,220077,122,53,57,0,0.0,0.0,0.0,0,0.0,0.0,0.0,232,0.0,0.0,0.0,0.0
3,2025-03-09,Haryana,FaridabadGurugram,243005,98,79,23,0,0.0,0.0,0.0,0,0.0,0.0,0.0,200,0.0,0.0,0.0,0.0
4,2025-03-09,Karnataka,Bengaluru UrbanBengaluru UrbanBengaluru UrbanB...,2240159,63,80,105,0,0.0,0.0,0.0,0,0.0,0.0,0.0,248,0.0,0.0,0.0,0.0


In [ ]:
merged.describe()


,date,pincode_x,age_0_5,age_5_17,age_18_greater,pincode_y,demo_age_5_17,demo_age_17_,pincode,bio_age_5_17,bio_age_17_,total_enrolment,demographic_updates,biometric_updates,DUI,BUBI
count,3393,3.393000e+03,3393.000000,3393.000000,3393.000000,3.393000e+03,3393.000000,3393.000000,3.393000e+03,3393.000000,3393.000000,3393.000000,3393.000000,3.393000e+03,3393.000000,3251.000000
mean,2025-10-14 21:16:36.286472192,1.537779e+08,1045.377247,507.039198,49.617742,2.375909e+08,911.402594,7899.001768,2.181687e+08,7516.252874,7894.153257,1602.034188,8810.404362,1.541041e+04,15.964878,69.658072
min,2025-03-02 00:00:00,1.000000e+05,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,2025-09-17 00:00:00,3.720622e+06,13.000000,2.000000,0.000000,2.254343e+06,2.000000,17.000000,1.483840e+06,3.000000,5.000000,19.000000,20.000000,9.000000e+00,1.789552,0.000000
50%,2025-10-21 00:00:00,2.690508e+07,122.000000,48.000000,2.000000,3.670954e+07,73.000000,473.000000,3.494671e+07,211.000000,204.000000,224.000000,542.000000,4.210000e+02,4.813934,14.356135
75%,2025-11-12 00:00:00,2.768620e+08,1223.000000,285.000000,19.000000,3.370621e+08,863.000000,7582.000000,3.277737e+08,3911.000000,4613.000000,1592.000000,8530.000000,8.756000e+03,9.182692,44.759871
max,2025-12-31 00:00:00,1.275220e+09,73166.000000,81078.000000,8768.000000,2.230183e+09,65004.000000,352128.000000,2.262708e+09,902096.000000,776958.000000,157866.000000,379443.000000,1.426001e+06,7616.500000,14705.333333
std,NaN,2.334632e+08,2554.154220,2229.347176,319.347286,3.938166e+08,2724.534871,20736.516023,3.591236e+08,39917.662317,39332.293533,4780.361617,23123.615836,7.655478e+04,161.618778,482.343735


In [ ]:
PROCESSED_PATH = "/content/drive/MyDrive/uidai_hackathon/data/processed"
os.makedirs(PROCESSED_PATH, exist_ok=True)

merged.to_csv(f"{PROCESSED_PATH}/lifecycle_aggregated.csv", index=False)
print("Saved lifecycle_aggregated.csv")


Saved lifecycle_aggregated.csv
